In [10]:
import pandas as pd
from pathlib import Path
import re
import warnings
import sys
import os
sys.path.append(os.path.join(sys.path[0], '../src'))

from data_preprocessor import preprocess_data

warnings.filterwarnings("ignore")

In [11]:
def print_sample(sample):
    """Print one sample from the entire dataset."""

    print(f"Context:{sample['context']}")
    try:
        print(f"Question:{sample['Question']}")
    except KeyError:
        print(f"Question:{sample['question']}")
    try:
        print(f"Answer:{sample['Answer']}")
    except KeyError:
        print(f"Answer:{sample['answer']}")

In [12]:
BASE_DIR = Path('EDA_MedQuAD_generated.ipynb').resolve().parent.parent
DATA_DIR = BASE_DIR.joinpath('data')
FILE_NAME = "MedQA_generated_Russian.csv"

FILE = DATA_DIR.joinpath(FILE_NAME)

In [13]:
data = pd.read_csv(FILE, sep=",")

print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5198 entries, 0 to 5197
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5198 non-null   int64 
 1   context                 5196 non-null   object
 2   question                5198 non-null   object
 3   original_context_index  5198 non-null   int64 
 4   answer                  5198 non-null   object
dtypes: int64(2), object(3)
memory usage: 203.2+ KB
None


,Unnamed: 0,context,question,original_context_index,answer
0,0,Синдром поликистозных яичников (СПКЯ) – это со...,"Как называется заболевание, вызывающее повышен...",0,андрогены
1,1,Причины этих изменений неясны. Затрагиваемые г...,Какие два гормона помогают яичникам женщины вы...,0,Эстроген и прогестерон
2,2,скопление жидкости (кисты) вокруг них. Поражен...,Какова овуляция у женщины с СПКЯ?,0,овуляция происходит не каждый месяц
3,3,у девочки начинаются месячные. У женщин с этим...,Как называется симптом СПКЯ?,0,соски
4,4,в виде темных или толстых пятен на коже и скла...,Какова частая причина облысения у мужчин?,0,волосы на голове у висков
5,5,Показать: Увеличенные яичники со множеством ме...,Что является распространенным заболеванием у ж...,0,уровень гормонов
6,6,Может быть проведено: определение уровня глюко...,Какие тесты можно сдать для проверки неперенос...,0,содержание сахара в крови
7,7,"противозачаточные таблетки, чтобы сделать менс...",Какое лекарство от диабета может помочь вам сд...,0,Глюкофаж
8,8,"Регулярные препараты, которые помогут вам забе...",Какие еще лекарства могут помочь вам заберемен...,0,электроэпиляция и лазерная эпиляция
9,9,может потребоваться любое лечение. Лечение дор...,Каков риск эндометриоза?,0,Во время беременности


In [14]:
print_sample(data.iloc[0])

Context:Синдром поликистозных яичников (СПКЯ) – это состояние, при котором у женщины повышается уровень мужских гормонов (андрогенов). В результате повышения уровня гормонов возникает множество проблем, в том числе: Нарушения менструального цикла. Бесплодие. Проблемы с кожей, такие как прыщи и повышенный рост волос. Увеличение количества мелких кист в яичниках. Устойчивость к инсулину с повышенным риском развития диабета. Причины СПКЯ связаны с изменениями уровня гормонов, которые затруднить выход яичников полностью выращенных (зрелых) яйцеклеток.
Question:Как называется заболевание, вызывающее повышение уровня мужских гормонов?
Answer:андрогены


In [15]:
print_sample(data.iloc[10])

Context:рак БесплодиеДиабет Осложнения, связанные с ожирением Когда обращаться к медицинскому работнику Если у вас есть симптомы этого расстройства, обратитесь к своему врачу.
Question:Как называется состояние, которое может вызвать у вас проблемы с весом?
Answer:рак


In [16]:
print_sample(data.iloc[15])

Context:Профессионал Это состояние можно обнаружить во время раннего осмотра младенца. Для диагностики синдрома Нунан часто необходим генетик. Профилактика Пары с семейным анамнезом синдрома Нунан, возможно, захотят рассмотреть возможность генетического консультирования, прежде чем заводить детей. Изображения Pectus excavatum
Question:Как называется состояние, которое часто обнаруживается при раннем осмотре ребенка?
Answer:Воронкообразная деформация грудной клетки


In [17]:
print_sample(data.iloc[100])

Context:Описание Синдром кольцевой хромосомы 20 — это состояние, которое влияет на нормальное развитие и функцию мозга. Наиболее распространенной особенностью этого состояния являются повторяющиеся припадки (эпилепсия) в детстве. Приступы могут возникать днем ​​или ночью во время сна. Их называют парциальными припадками, поскольку они затрагивают только одну область мозга, область, называемую лобной долей. Во многих случаях припадки носят сложный характер и резистентны к лечению противоэпилептическими препаратами. Длительные приступы
Question:Как называется состояние, которое влияет на нормальное развитие и функцию мозга?
Answer:Синдром кольцевой хромосомы 20


### Data Preprocessing
1. Construct the dataset with columns: ["question", "context", "answer_text", "answer_start"]
2. Check all samples with -1 position
3. Remove all occurences 'Вопрос:' from the "Question" column
4. Remove all occurences 'Резюме:' and 'Краткое описание:' from the "Answer" column
5. Remove all samples with context "На этой странице Основные сведения Начать здесь Диагностика и тесты Узнайте больше Сопутствующие вопросы Подробности Смотрите, играйте и учитесь Изображения Статистика исследований и исследования Клинические испытания Журнальные статьи"
6. Remove all '\n, \t, -, —' and extra spaces
7. Lowercase
8. Remove all duplicates: ["question", "context"]

In [19]:
data = data[["question", "context", "answer"]]
data

,question,context,answer
0,"Как называется заболевание, вызывающее повышен...",Синдром поликистозных яичников (СПКЯ) – это со...,андрогены
1,Какие два гормона помогают яичникам женщины вы...,Причины этих изменений неясны. Затрагиваемые г...,Эстроген и прогестерон
2,Какова овуляция у женщины с СПКЯ?,скопление жидкости (кисты) вокруг них. Поражен...,овуляция происходит не каждый месяц
3,Как называется симптом СПКЯ?,у девочки начинаются месячные. У женщин с этим...,соски
4,Какова частая причина облысения у мужчин?,в виде темных или толстых пятен на коже и скла...,волосы на голове у висков
...,...,...,...
5193,Как по-другому называется KWWH?,"он мутировал ген, но у них обычно не проявляют...",КВВХ
5194,"Как называется состояние, вызывающее крайнюю б...",Описание Синдром Кноблоха — редкое заболевание...,высокая близорукость
5195,Какая макула отвечает за остроту центрального ...,сетчатка — светочувствительная ткань в задней ...,макула
5196,Что такое затылочное энцефалоцеле?,Элопная слепота на один или оба глаза. Другой ...,мешковидное выпячивание головного мозга


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5198 entries, 0 to 5197
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  5198 non-null   object
 1   context   5196 non-null   object
 2   answer    5198 non-null   object
dtypes: object(3)
memory usage: 122.0+ KB


In [22]:
# Remove all rows with NaNs
data.dropna(axis=0, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5196 entries, 0 to 5197
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  5196 non-null   object
 1   context   5196 non-null   object
 2   answer    5196 non-null   object
dtypes: object(3)
memory usage: 162.4+ KB


There is no meaningful context

In [23]:
# Remove all samples with insignificant context
ids = []
substr = "На этой странице"
for i in range(data.shape[0]):
    if substr in data.iloc[i]["context"]:
        ids.append(i)
data_tmp2 = data.iloc[ids,:]

data_tmp2

,question,context,answer
66,Что представляет собой главная страница сводки...,На этой странице Краткое изложение основ Начни...,Краткое изложение основ Начните здесь
107,"Как называется страница, содержащая основную и...",На этой странице Основные сведения Начать здес...,Краткое изложение основ
144,Какова основная тема Краткого обзора?,На этой странице Основные сведения Начать здес...,Диагностика и тесты
203,Каково основное резюме Основ генетики?,На этой странице Краткое изложение основ Начни...,Краткое изложение основ
279,Каково основное резюме книги «Жизнь со связанн...,На этой странице Краткое изложение основ Начни...,Краткое изложение основ
...,...,...,...
4880,"Как называется страница, содержащая основную и...",На этой странице Основные сведения Начать здес...,Краткое изложение основ Начните здесь
4881,"Как называется страница, содержащая основную и...",На этой странице Основные сведения Начать здес...,Краткое изложение основ Начните здесь
4882,Какова основная краткая информация о методах л...,На этой странице Краткое описание основ Начнит...,Краткое изложение основ
4916,Какова основная тема Краткого изложения основ?,На этой странице Основные сведения Начать здес...,Статьи журнала клинических исследований


In [25]:
data_tmp2.loc[107]["context"]

'На этой странице Основные сведения Начать здесь Диагностика и тесты Узнайте больше Сопутствующие вопросы Подробности Смотрите, играйте и учитесь Нет доступных ссылок Исследования Клинические испытания Журнальные статьи'

In [20]:
data_tmp2.loc[2262]["context"]

'На этой странице Краткое описание Основы Начните здесь Узнайте больше Вопросы по теме Особенности Смотрите, играйте и учитесь Изображения Исследования Клинические исследования Журнальные статьи'

In [26]:
# Remove all samples with insignificant context
ids = []
substr = "Эти ресурсы"
for i in range(data.shape[0]):
    if substr in data.iloc[i]["answer"]:
        ids.append(i)
data_tmp2 = data.iloc[ids,:]

data_tmp2

,question,context,answer


## Data preprocessing

In [27]:
data_example = " что такое диарея – обзор? также называется: стул водянистый"
data_example = re.sub('[-–—\n\t\(\)\[\]]', '', data_example)
data_example = re.sub(' +', ' ', data_example)
data_example

' что такое диарея обзор? также называется: стул водянистый'

### Data Preprocessing for the entire dataframe

In [28]:
data = preprocess_data(data)
data

,question,context,answer
0,"как называется заболевание, вызывающее повышен...",синдром поликистозных яичников спкя это состоя...,андрогены
1,какие два гормона помогают яичникам женщины вы...,причины этих изменений неясны. затрагиваемые г...,эстроген и прогестерон
2,какова овуляция у женщины с спкя?,скопление жидкости кисты вокруг них. пораженны...,овуляция происходит не каждый месяц
3,как называется симптом спкя?,у девочки начинаются месячные. у женщин с этим...,соски
4,какова частая причина облысения у мужчин?,в виде темных или толстых пятен на коже и скла...,волосы на голове у висков
...,...,...,...
5193,как подругому называется kwwh?,"он мутировал ген, но у них обычно не проявляют...",кввх
5194,"как называется состояние, вызывающее крайнюю б...","синдром кноблоха редкое заболевание, характер...",высокая близорукость
5195,какая макула отвечает за остроту центрального ...,сетчатка светочувствительная ткань в задней ча...,макула
5196,что такое затылочное энцефалоцеле?,элопная слепота на один или оба глаза. другой ...,мешковидное выпячивание головного мозга


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5109 entries, 0 to 5197
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  5109 non-null   object
 1   context   5109 non-null   object
 2   answer    5109 non-null   object
dtypes: object(3)
memory usage: 159.7+ KB


In [30]:
data.groupby(["question", "context"])["answer"].count()

question                                                                                                                                context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
альтернирующая гемиплегия детского возраста неврологическое состояние, характеризующееся повторяющимися эпизодами временного паралича?   альтернирующая гемиплегия детского возраста неврологическое состояние, характеризу

In [31]:
# Remove duplicates
data.drop_duplicates(subset=['question', 'context'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5084 entries, 0 to 5183
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  5084 non-null   object
 1   context   5084 non-null   object
 2   answer    5084 non-null   object
dtypes: object(3)
memory usage: 158.9+ KB


### Save Dataset

In [32]:
DATA_NAME = "MedQuAD_generated_clean.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data.to_csv(FILE)

In [33]:
# Concatenate two datasets
DATA_NAME = "MedQuAD_answers_augmented_concat.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data2 = pd.read_csv(FILE, index_col=0)

data2

,question,context,answer
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю..."
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...
4,что такое синдром кноблоха?,"синдром кноблоха редкое заболевание, характер...","синдром кноблоха редкое заболевание, характери..."
...,...,...,...
1317,какие генетические изменения связаны с атаксие...,атаксия фридрейха это генетическое заболевани...,мутации в гене fxn вызывают атаксию фридрейха...
1318,что такое являются редкими заболеваниями?,"редкое заболевание это заболевание, которым с...",более 25 миллионов американцев имеют один. сущ...
1319,что такое являются гранулематоз с полиангиитом?,"гранулематоз с полиангиитом гпа, ранее извест...",гранулематоз вегенера редкое заболевание. вос...
1320,что такое мальформация киари?,мальформации киари мк представляют собой стру...,км это структурные дефекты мозжечка. часть че...


In [34]:
data = pd.concat([data2, data])
data

,question,context,answer
0,что такое кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами представляет...
1,сколько людей с шерстяными волосами страдают к...,кератодермия с пушистыми волосами представляе...,кератодермия с пушистыми волосами встречается ...
2,"каковы генетические изменения, связанные с кер...",кератодермия с пушистыми волосами представляе...,"мутации в генах jup, dsp, dsc2 и kank2 вызываю..."
3,передается ли кератодермия с пушистыми волосами?,кератодермия с пушистыми волосами представляе...,большинство случаев кератодермии с курчавыми в...
4,что такое синдром кноблоха?,"синдром кноблоха редкое заболевание, характер...","синдром кноблоха редкое заболевание, характери..."
...,...,...,...
5179,что такое гипотония?,"g часов, а также часто нарушают сон. непроизво...",слабый мышечный тонус
5180,"какова основная причина дискинезии, связанной ...","у некоторых людей с дискинезией, связанной с a...",беспокойство
5181,"какова причина дискинезии, связанной с adcy5?","кинезия, в то время как у некоторых больных мо...",церебральный паралич или эпилепсия
5182,"какова причина дискинезии, связанной с adcy5?","название предполагает, что дискинезия, связанн...",ген adcy5


In [35]:
data.groupby(["question", "context"])["answer"].count()

question                                                                                                                                                                     context                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [36]:
DATA_NAME = "MedQuAD_generated_concat.csv"
FILE = DATA_DIR.joinpath(DATA_NAME)

data.to_csv(FILE)